In [1]:
import pandas as pd

# 读取 Excel 文件
excel_file = "emissions.xlsx"

# 获取所有 sheet 名字
sheet_names = pd.ExcelFile(excel_file).sheet_names

# 依次读取并打印每个 sheet 的名字和内容
for sheet in sheet_names:
    print(f"Sheet name: {sheet}")
    df = pd.read_excel(excel_file, sheet_name=sheet)
    print(df)
    print("\n" + "="*40 + "\n")


Sheet name: FE_emission
    Unnamed: 0      ANHU      BEIJ      CHON      FUJI      GANS      GUAD  \
0         2020  2.013547  0.587034  1.117267  1.486283  0.853643  3.276580   
1         2021  1.984903  0.612617  1.120212  1.529369  0.853823  3.081785   
2         2022  1.924324  0.520964  1.042890  1.689208  0.869817  2.969463   
3         2023  2.137066  0.613015  1.176093  1.636356  0.908834  3.249238   
4         2024  2.160842  0.609427  1.190095  1.653611  0.915279  3.280373   
5         2025  2.164423  0.606331  1.196871  1.659610  0.913209  3.300155   
6         2026  2.173344  0.603496  1.205937  1.666617  0.913268  3.319930   
7         2027  2.182766  0.599683  1.213920  1.673990  0.913223  3.340724   
8         2028  2.193898  0.595946  1.223213  1.682448  0.913737  3.366026   
9         2029  2.184099  0.575689  1.210692  1.668616  0.904992  3.327674   
10        2030  2.175231  0.555846  1.198756  1.655470  0.896681  3.291704   
11        2031  2.110216  0.531024  1.16

In [ ]:
import pandas as pd
import json

# --- 1. CONFIGURACIÓN ---
excel_file = "emissions.xlsx"
output_json_file = "emissions.json"

# Mapeo de los nombres de las hojas a las claves que queremos en el JSON
sheet_map = {
    'FE_emission': 'FE',
    'Emi_supply': 'SUPPLY',
    'Emi_total': 'TOTAL'
}

# Definir los años de interés: de 2025 a 2060, en pasos de 5 años.
years_to_keep = list(range(2025, 2061, 5))

# --- 2. LECTURA Y PREPARACIÓN DE DATOS ---

# Cargar cada hoja en un DataFrame de pandas
# Usamos 'index_col=0' para que la primera columna ('Unnamed: 0') se convierta en el índice (los años)
try:
    df_fe = pd.read_excel(excel_file, sheet_name='FE_emission', index_col=0)
    df_supply = pd.read_excel(excel_file, sheet_name='Emi_supply', index_col=0)
    df_total = pd.read_excel(excel_file, sheet_name='Emi_total', index_col=0)
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{excel_file}'. Asegúrate de que esté en el mismo directorio.")
    exit()

# Crear un diccionario para acceder fácilmente a los dataframes
dataframes = {
    'FE': df_fe,
    'SUPPLY': df_supply,
    'TOTAL': df_total
}

# Obtener una lista consistente de todas las provincias (columnas)
# Usamos la intersección para asegurarnos de que solo procesamos provincias presentes en las 3 hojas
provinces = list(set(df_fe.columns) & set(df_supply.columns) & set(df_total.columns))
provinces.sort() # Ordenar alfabéticamente para un resultado consistente

# --- 3. CONSTRUCCIÓN DE LA ESTRUCTURA JSON ---

# Diccionario principal que contendrá todos los datos
final_data = {}

# Iterar sobre cada provincia
for province in provinces:
    # Crear una entrada para la provincia actual
    final_data[province] = {}
    
    # Iterar sobre los tipos de emisión (FE, SUPPLY, TOTAL)
    for key, df in dataframes.items():
        # Filtrar el DataFrame por los años de interés y seleccionar la columna de la provincia
        # El resultado es una Serie de Pandas (años como índice, valores de emisión como datos)
        # Luego, convertir esta Serie a un diccionario {año: valor}
        province_year_data = df.loc[years_to_keep, province].to_dict()
        
        # Asignar el diccionario de {año: valor} a la clave correspondiente (FE, SUPPLY, o TOTAL)
        final_data[province][key] = province_year_data

# --- 4. ESCRITURA DEL ARCHIVO JSON ---
with open(output_json_file, 'w', encoding='utf-8') as f:
    # json.dump escribe el diccionario en el archivo
    # indent=4 formatea el JSON para que sea fácil de leer por humanos
    json.dump(final_data, f, ensure_ascii=False, indent=4)

print(f"¡Proceso completado! Los datos han sido guardados en el archivo: '{output_json_file}'")

# Opcional: Imprimir una muestra para verificar la estructura
print("\n--- Muestra de los datos para la provincia 'ANHU' ---")
print(json.dumps({'ANHU': final_data.get('ANHU', {})}, indent=4))